In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
import my_txtutils
import re
import nltk
mypath='./new_txts/'
txts='./txts/'

In [13]:
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(txts) if isfile(join(txts, f))]
print(onlyfiles)

['acts_new.txt', 'gal_eph_new.txt', 'heb_new.txt', 'jam_jud_new.txt', 'john_new.txt', 'jud_rev_new.txt', 'luke_8_john_new.txt', 'mark01_new.txt', 'matt02_new.txt', 'matt_new.txt', 'phil_col_new.txt', 'thes_tim_new.txt', 'tit_phl_new.txt']


In [5]:
#large files
def large_f():
    #filenames = ['file1.txt', 'file2.txt', ...]
    with open(mypath+'full.txt', 'w') as outfile:
        for fname in onlyfiles:
            with open(fname) as infile:
                for line in infile:
                    outfile.write(line)
#large files
def decimate_f(f):
    #filenames = ['file1.txt', 'file2.txt', ...]
    with open(mypath+'tmp.txt', 'w') as outfile:
        with open(mypath+f) as infile:
            lc=0
            for line in infile:
                wrap=False
                cx=0
                tmp=""
                for i in line:
                    wrap=True if cx>99 else False
                    if (wrap==True and i==' '):
                        tmp+='\n'
                        cx=-1
                    else:
                        tmp+=i
                    cx+=1
                outfile.write(tmp)
                lc+=1
    print(lc,'lines written')
decimate_f('full.txt')

311 lines written


In [22]:
#small files
def small_f():
    #filenames = ['file1.txt', 'file2.txt', ...]
    with open(mypath+'full.txt', 'w') as outfile:
        for fname in onlyfiles:
            print('writing file',fname)
            with open(txts+fname) as infile:
                outfile.write(infile.read())
    return open(mypath+"full.txt").read()

In [24]:
txt=small_f()
words=re.sub(' +', ' ',txt.replace('\n',' ')).split(' ')
len(words)

writing file acts_new.txt
writing file gal_eph_new.txt
writing file heb_new.txt
writing file jam_jud_new.txt
writing file john_new.txt
writing file jud_rev_new.txt
writing file luke_8_john_new.txt
writing file mark01_new.txt
writing file matt02_new.txt
writing file matt_new.txt
writing file phil_col_new.txt
writing file thes_tim_new.txt
writing file tit_phl_new.txt


649190

In [25]:
len(txt)

6507438

In [4]:
# these must match what was saved !
ALPHASIZE = my_txtutils.ALPHASIZE
NLAYERS = 3
INTERNALSIZE = 512

ok02 ="checkpoints/rnn_train_1560839489-30000000" # okrika 2019-06-18
ok03 ="checkpoints/rnn_train_1563601394-60000000" # okrika 2019-07-20
bb01 ="checkpoints/rnn_train_1563814502-18000000" # bible 2019-07-22

author=bb01

ncnt = 0
with tf.Session() as sess:
    # new_saver = tf.train.import_meta_graph('checkpoints/rnn_train_1512567262-0.meta')
    new_saver = tf.train.import_meta_graph(author+'.meta')
    new_saver.restore(sess, author)
    x = my_txtutils.convert_from_alphabet(ord("L"))
    x = np.array([[x]])  # shape [BATCHSIZE, SEQLEN] with BATCHSIZE=1 and SEQLEN=1

    # initial values
    y = x
    h = np.zeros([1, INTERNALSIZE * NLAYERS], dtype=np.float32)  # [ BATCHSIZE, INTERNALSIZE * NLAYERS]
    for i in range(1000):
        yo, h = sess.run(['Yo:0', 'H:0'], feed_dict={'X:0': y, 'pkeep:0': 1., 'Hin:0': h, 'batchsize:0': 1})
        #print(tf.shape(yo))
        #print(len(yo))
        c = my_txtutils.sample_from_probabilities(yo, topn=2)
        y = np.array([[c]])  # shape [BATCHSIZE, SEQLEN] with BATCHSIZE=1 and SEQLEN=1
        c = chr(my_txtutils.convert_to_alphabet(c))
        print(c, end="")
        if c == '\n':
            ncnt = 0
        else:
            ncnt += 1
        if ncnt == 100:
            print("")
            ncnt = 0


INFO:tensorflow:Restoring parameters from checkpoints/rnn_train_1563814502-18000000
OODRDD,   s a y i n g , 
 T h e   s e n   o f   I s r a e l ,   s a y i n g ,   T h e   L O R D   h a t h   s e e n   t h e e
   t o   p a s s   t h e   L O R D ,   t h a t   t h e   L O R D   s h a l l   b e   u n c l e a n .
 
 A n d   t h e   s a m e   s a w   t h e r e o f   s h a l l   b e   u n c l e a n . 
 A n d   t h e   L O R D   s a i d ,   T h u s   s a i t h   t h e   L O R D ,   t h a t   t h e y  
 s h a l l   b r i n g   t h y   s o u l   w i t h   t h e   s t r a n g e r s ,   a n d   t o   s p
 e a k ,   t o   s a y   u n t o   t h e m ,   W h a t   s a y   u n t o   t h e e   t h e   L O R D
 ,   a n d   t h e   L O R D   s h o u l d   s a y   u n t o   h i m ,   T h e   L O R D   s p a k e
   u n t o   h i m ,   W h o   h a t h   s e n t   t o   p a s s   i n t o   t h e   s e r v a n t ,
   a n d   t o   s e r v e   t h e   c h i e f   o f f e r i n g ,   t h e   L O R D   s h a l l

In [18]:
def pcx(txt):
    # these must match what was saved !
    ALPHASIZE = my_txtutils.ALPHASIZE
    NLAYERS = 3
    INTERNALSIZE = 512

    ok03 ="checkpoints/rnn_train_1563601394-60000000" # okrika 2019-07-20
    bb01 ="checkpoints/rnn_train_1563814502-18000000" # bible 2019-07-22

    author=bb01

    ncnt = 0
    with tf.Session() as sess:
        # new_saver = tf.train.import_meta_graph('checkpoints/rnn_train_1512567262-0.meta')
        new_saver = tf.train.import_meta_graph(author+'.meta')
        new_saver.restore(sess, author)
        x = my_txtutils.convert_from_alphabet(ord(txt[0]))
        x = np.array([[x]])  # shape [BATCHSIZE, SEQLEN] with BATCHSIZE=1 and SEQLEN=1

        # initial values
        y = x
        h = np.zeros([1, INTERNALSIZE * NLAYERS], dtype=np.float32)  # [ BATCHSIZE, INTERNALSIZE * NLAYERS]
        arr=[]
        for i in range(len(txt)-1):
            yo, h = sess.run(['Yo:0', 'H:0'], feed_dict={'X:0': y, 'pkeep:0': 1., 'Hin:0': h, 'batchsize:0': 1})
            c = my_txtutils.convert_from_alphabet(ord(txt[i+1]))
            d = my_txtutils.sample_from_probabilities(yo, topn=1)
            d = chr(my_txtutils.convert_to_alphabet(d))
            y = np.array([[c]])  # shape [BATCHSIZE, SEQLEN] with BATCHSIZE=1 and SEQLEN=1
            y1=np.reshape(yo,[ALPHASIZE,-1])
            y1=y1.flatten()
            #c = chr(my_txtutils.convert_to_alphabet(i))
            print(txt[i],':',c,':',y1[c],',max=',np.max(y1),"='%s' or %s"%(chr(my_txtutils.convert_to_alphabet(np.argmax(y1))),d))
            arr.append(y1[c])
    return np.log2(np.sum(arr))*(-1/len(txt))
    #return np.prod(arr))**(-1/len(txt)
t='''I will say unto his son, Thou shalt
 not be saved, that I say unto them, We have consu
med my father, and the mighty man that shall be un
clean unto him, and the LORD hath said, I will pro
voke me to me, and the remembring of the children 
of Israel saw the LORD'''
pcx(t)

INFO:tensorflow:Restoring parameters from checkpoints/rnn_train_1563814502-18000000
I : 2 : 0.0 ,max= 0.021843001 =' ' or  
  : 89 : 0.0 ,max= 0.03725405 ='s' or s
w : 75 : 0.0 ,max= 0.08176538 ='s' or s
i : 78 : 0.0 ,max= 0.15770254 =' ' or  
l : 78 : 0.0 ,max= 0.26536465 ='s' or s
l : 2 : 0.0 ,max= 0.9873104 =' ' or  
  : 85 : 0.0 ,max= 0.15239526 ='t' or t
s : 67 : 0.0 ,max= 0.15094186 =' ' or  
a : 91 : 0.0 ,max= 0.8122729 =' ' or  
y : 2 : 0.45777845 ,max= 0.45777845 =' ' or  
  : 87 : 0.0 ,max= 0.9496611 =' ' or  
u : 80 : 0.2012075 ,max= 0.2012075 ='n' or n
n : 86 : 0.0 ,max= 0.66236836 ='n' or n
t : 81 : 0.0 ,max= 0.9999535 =' ' or  
o : 2 : 0.0 ,max= 0.35473773 ='t' or t
  : 74 : 0.0 ,max= 0.99999845 =' ' or  
h : 75 : 0.0 ,max= 0.39989376 ='o' or o
i : 85 : 0.0 ,max= 0.9999908 =' ' or  
s : 2 : 0.0 ,max= 0.7822604 ='n' or n
  : 85 : 0.0 ,max= 0.9999951 =' ' or  
s : 81 : 0.0 ,max= 0.2110508 ='t' or t
o : 80 : 0.0 ,max= 0.9999993 =' ' or  
n : 14 : 0.0 ,max= 0.83871084 ='n' or

-0.007018808645146493

In [43]:
def pcx2(txt):
    # these must match what was saved !
    ALPHASIZE = my_txtutils.ALPHASIZE
    NLAYERS = 3
    INTERNALSIZE = 512

    bb01 ="checkpoints/rnn_train_1563814502-18000000" # bible 2019-07-22
    ok03 ="checkpoints/rnn_train_1563601394-60000000" # okrika 2019-07-20

    author=bb01

    ncnt = 0
    with tf.Session() as sess:
        # new_saver = tf.train.import_meta_graph('checkpoints/rnn_train_1512567262-0.meta')
        new_saver = tf.train.import_meta_graph(author+'.meta')
        new_saver.restore(sess, author)
        x = my_txtutils.convert_from_alphabet(ord(txt[0]))
        x = np.array([[x]])  # shape [BATCHSIZE, SEQLEN] with BATCHSIZE=1 and SEQLEN=1

        # initial values
        y = x
        h = np.zeros([1, INTERNALSIZE * NLAYERS], dtype=np.float32)  # [ BATCHSIZE, INTERNALSIZE * NLAYERS]
        fi=se=th=rn=txt[0]
        arr=[]
        for i in range(len(txt)-1):
            yo, h = sess.run(['Yo:0', 'H:0'], feed_dict={'X:0': y, 'pkeep:0': 1., 'Hin:0': h, 'batchsize:0': 1})
            c = my_txtutils.convert_from_alphabet(ord(txt[i+1]))
            d = my_txtutils.sample_from_probabilities(yo, topn=2)
            d = chr(my_txtutils.convert_to_alphabet(d))
            y = np.array([[c]])  # shape [BATCHSIZE, SEQLEN] with BATCHSIZE=1 and SEQLEN=1
            y1=np.reshape(yo,[ALPHASIZE,-1])
            y1=y1.flatten()
            y2=y1.argsort()[-3:][::-1]
            fi+=chr(my_txtutils.convert_to_alphabet(y1[y2[0]]))
            se+=chr(my_txtutils.convert_to_alphabet(y1[y2[1]]))
            th+=chr(my_txtutils.convert_to_alphabet(y1[y2[2]]))
            rn+=d
            #c = chr(my_txtutils.convert_to_alphabet(i))
            #print(txt[0:i+1],'|',fi,'|',se,'|',th,'|',rn)
            arr.append(y1[c])
    print(txt,'\n')
    print(fi,'\n')
    print(se,'\n')
    print(th,'\n')
    print(rn,'\n')
    return np.log2(np.sum(arr))*(-1/len(txt))
    #return np.prod(arr))**(-1/len(txt)
t='''I will say unto his son, Thou shalt
 not be saved, that I say unto them, We have consu
med my father, and the mighty man that shall be un
clean unto him, and the LORD hath said, I will pro
voke me to me, and the remembring of the children 
of Israel saw the LORD'''
pcx2(t)

INFO:tensorflow:Restoring parameters from checkpoints/rnn_train_1563814502-18000000
I will say unto his son, Thou shalt
 not be saved, that I say unto them, We have consu
med my father, and the mighty man that shall be un
clean unto him, and the LORD hath said, I will pro
voke me to me, and the remembring of the children 
of Israel saw the LORD 

I                                   	 	                                                 	   	                     	     	                         	 	                 	     	               	                                                     	   	                

I                                                                                                                                                                                                                                                                      

I                                                                                                                        

-0.00962196597616181

In [10]:
print('yes' if re.search("g", "abcdef") else 'no')

no


In [46]:
my_txtutils.convert_from_alphabet(ord('i'))

75

In [26]:
a=np.array([1,3,5,0])
b=dict(zip([x for x in range(len(a))],a))

In [36]:
c='hello'

In [37]:
c[0:2]

'he'